# 02: Basic Operations and Analysis

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Austfi/xsnowForPatrol/blob/main/notebooks/02_basic_operations_and_analysis.ipynb)

Now that you understand xsnow's data structure, let's learn how to work with the data: selecting, filtering, and performing basic analyses.

## What You'll Learn

- Selecting data by location, time, and layer
- Filtering data with conditions
- Computing profile-level summaries
- Calculating snow water equivalent (SWE)
- Identifying weak layers
- Time series operations


## Installation (For Colab Users)

If you're using Google Colab, run the cell below to install xsnow and dependencies. If you're running locally and have already installed xsnow, you can skip this cell.


In [ ]:

%pip install -q numpy pandas xarray matplotlib seaborn dask netcdf4
%pip install -q git+https://gitlab.com/avacollabra/postprocessing/xsnow



## Setup: Load Sample Data

Let's load sample data to work with:


In [ ]:
import xsnow
import numpy as np

# Load sample data
print("Loading xsnow sample data...")
try:
    ds = xsnow.single_profile_timeseries()
    print("✅ Data loaded successfully")
    print(f"Dimensions: {dict(ds.dims)}")
except Exception as e:
    print(f"⚠️ Error loading sample data: {e}")
    print("Make sure xsnow is properly installed:")
    print("  pip install git+https://gitlab.com/avacollabra/postprocessing/xsnow")
    raise


## Part 1: Selecting Data

You can select parts of your data in two ways:

### Selecting by Label with `.sel()`

Use `.sel()` when you know the label (like a date or location name):

**What you'll see:** The code below selects data for a specific location and time. The result is a smaller dataset containing only the selected data.


In [ ]:
# Select a specific location
location_name = ds.coords['location'].values[0]
ds_site = ds.sel(location=location_name)

# Select a specific time (or time range)
times = ds.coords['time'].values

# Select a single time
ds_single_time = ds.sel(time=times[0])

# Select a time range using slice
ds_time_range = ds.sel(time=slice(times[0], times[9]))


### Selecting by Index with `.isel()`

Use `.isel()` when you want to select by position (first, second, etc.):

**What you'll see:** These examples show how to select the first time step, first few time steps, surface layer, and multiple layers. The resulting datasets will have fewer dimensions than the original.


In [ ]:
# Get first time step
ds_first = ds.isel(time=0)

# Get first 5 time steps
ds_first5 = ds.isel(time=slice(0, 5))

# Get surface layer (layer 0)
surface = ds.isel(layer=0)

# Get multiple layers
top_layers = ds.isel(layer=[0, 1, 2])  # Top 3 layers


### 🎯 Fill in the Blanks: Selection Practice

**Fill in the Blanks Pattern**: Complete the missing code below. Replace `???` with the correct method or parameter:


In [ ]:
# TODO: Fill in the missing method to select the first location by label
location_name = ds.coords['location'].values[0]
ds_site = ds.???(location=location_name)

# TODO: Fill in the missing method to select the first time step by index
ds_first = ds.???(time=0)

# TODO: Fill in the missing dimension name to select the surface layer
surface = ds.isel(???=0)

# TODO: Fill in the missing parameter to select layers 0, 1, and 2
top_layers = ds.isel(layer=[0, 1, 2])  # This one is already done - use it as a hint!


### Selecting by Depth (using 'z' coordinate)

Remember the `z` coordinate? We can use it to select layers by depth:

**What you'll see:** The code below filters for layers within 50 cm of the surface. The result will have fewer layers than the original dataset, containing only layers that meet the depth criteria.


In [ ]:
# Select layers within 50 cm of surface (z between 0 and -0.5)
# Note: z is negative downward, so we want z >= -0.5
shallow = ds.where((ds.coords['z'] >= -0.5) & (ds.coords['z'] <= 0), drop=True)


## Part 2: Filtering Data with Conditions

You can filter data based on conditions (e.g., "show me only dense layers"). This is done with `.where()`:

**What you'll see:** The code below creates filtered datasets containing only layers that meet specific criteria (dense snow, cold temperatures). These filtered datasets will have fewer layers than the original.

**Now You Try**: After running the examples, try filtering for:
- Layers with temperature between -5°C and 0°C (hint: use `&` for "and", `|` for "or")
- Layers with density greater than 250 kg/m³


### 🎯 Fill in the Blanks: Filtering Practice

Complete the filtering code below:


In [ ]:
# TODO: Fill in the missing condition to find layers with temperature < -5°C
# Hint: Look at the dense_mask example above
cold_mask = ds['???'] < ???
cold_layers = ds.where(cold_mask, drop=True)

# TODO: Fill in the missing parameter to actually drop filtered-out values
# Hint: Look at the dense_layers example above
warm_layers = ds.where(ds['temperature'] > 0, ???=True)


In [ ]:
# Find layers with density > 300 kg/m³ (dense snow)
dense_mask = ds['density'] > 300
dense_layers = ds.where(dense_mask, drop=True)

# Find cold layers (temperature < -10°C)
cold_mask = ds['temperature'] < -10
cold_layers = ds.where(cold_mask, drop=True)


## Part 3: Computing Profile-Level Summaries

Often you want to summarize layer-level data into a single value per profile. For example, "what's the average density in each profile?"

**What you'll see:** The code below computes mean, max, and min density across all layers for each profile. The result is a new variable with the `layer` dimension removed - it's now a profile-level variable (one value per profile per time step).


In [ ]:
# Compute mean density across all layers for each profile
mean_density = ds['density'].mean(dim='layer')

# Other useful aggregations
max_density = ds['density'].max(dim='layer')  # Maximum density in profile
min_density = ds['density'].min(dim='layer')   # Minimum density in profile
std_density = ds['density'].std(dim='layer')   # Standard deviation


# Add as new variable to dataset
ds = ds.assign(mean_density=mean_density)


### 🎯 Fill in the Blanks: Aggregation Practice

Complete the aggregation code:


In [ ]:
# TODO: Fill in the missing dimension name to compute mean density across layers
mean_density = ds['density'].mean(dim='???')

# TODO: Fill in the missing aggregation method to find maximum temperature
max_temp = ds['temperature'].???(dim='layer')

# TODO: Fill in the missing aggregation method to find minimum density
min_density = ds['density'].???(dim='layer')


## Part 4: Calculating Snow Water Equivalent (SWE)

**Snow Water Equivalent (SWE)** is a critical metric: it tells you how much water is in the snowpack. SWE = density × thickness, summed over all layers.

### Understanding SWE

- **Why it matters**: SWE tells you the actual water content, not just snow depth
- **Units**: Typically mm or m of water
- **Calculation**: For each layer: density (kg/m³) × thickness (m) = water equivalent (kg/m² or mm)

Let's compute it:


In [ ]:
# Compute thickness from 'z' coordinate (depth)
z = ds.coords['z']
z_diff = z.diff(dim='layer', label='upper')
thickness = -z_diff  # Negative because z decreases downward

# For the last layer, estimate thickness from remaining depth
total_thickness = -z.isel(layer=0)  # Depth of first layer
computed_thickness = thickness.sum(dim='layer', skipna=True)
last_layer_thickness = total_thickness - computed_thickness
thickness = thickness.fillna(last_layer_thickness)

# Calculate SWE = sum(density * thickness) over all layers
swe = (ds['density'] * thickness).sum(dim='layer') / 1000.0  # Convert to m
ds = ds.assign(SWE=swe)
print(f"SWE calculated: {swe.values[0]:.3f} m (first profile)")


## Part 5: Identifying Weak Layers

Weak layers are critical for avalanche forecasting. A weak layer is typically characterized by:
- Low density
- Specific grain types (e.g., faceted crystals)
- Low strength/hardness

Let's create a simple weak layer identifier:


In [ ]:
# Define weak layer criteria
# Weak layers often have low density (< 150 kg/m³)
# This is a simplified example - real weak layer identification is more complex
weak_mask = ds['density'] < 150
weak_layers = ds.where(weak_mask, drop=True)
print(f"Found {weak_mask.sum().values} weak layer instances")


**Now You Try**: After computing SWE, try:
- Computing SWE for a different time period (hint: select a time range first)
- Comparing SWE between different locations
- Plotting SWE over time


## Part 6: Time Series Operations

xsnow makes it easy to analyze how snowpack properties change over time.

### Time-based Aggregations


In [ ]:
# Compute mean over time (average profile)
mean_density_over_time = ds['density'].mean(dim='time')

# Compute time series of profile-level variables
hs_series = ds['HS'].isel(location=0, slope=0, realization=0)

# Compute change over time (difference between consecutive time steps)
hs_change = ds['HS'].diff(dim='time')
print(f"Mean snow height change: {hs_change.mean().values:.4f} m/time step")


**Now You Try**: After identifying weak layers, try:
- Finding weak layers in a specific time period
- Counting how many weak layers exist at each time step
- Comparing weak layer density thresholds (try < 100, < 150, < 200)


### Accessing Values as NumPy Arrays

Sometimes you need raw NumPy arrays for integration with other libraries:


In [ ]:
# Get values as NumPy array
density_array = ds['density'].values

# Get values for a specific selection
surface_density = ds['density'].sel(layer=0).values


## 🎯 Target Practice: Selection Methods

**Target Practice Pattern**: Focused skill-building exercises. Complete these 5 quick exercises to master selection:

1. Select the last time step using `.isel()`
2. Select all data for the first location using `.sel()`
3. Select layers 0, 1, and 2 using `.isel()`
4. Select a time range from the first to the 10th time step using `.sel()` with `slice()`
5. Select the surface layer (layer 0) for all times


In [ ]:
# Target Practice Exercise 1: Select the last time step
# Your code here:
# ds_last = ds.???(time=???)

# Target Practice Exercise 2: Select all data for the first location
# Your code here:
# location_name = ds.coords['location'].values[0]
# ds_site = ds.???(location=???)

# Target Practice Exercise 3: Select layers 0, 1, and 2
# Your code here:
# top_3_layers = ds.???(layer=[0, 1, 2])

# Target Practice Exercise 4: Select time range from first to 10th time step
# Your code here:
# times = ds.coords['time'].values
# ds_range = ds.???(time=slice(???, ???))

# Target Practice Exercise 5: Select surface layer for all times
# Your code here:
# surface_all_times = ds.???(layer=0)


## 🎯 Target Practice: Filtering

Complete these filtering challenges:

1. Find layers with temperature between -5°C and 0°C
2. Find layers with density greater than 250 kg/m³
3. Find layers where temperature is below -10°C OR density is below 100 kg/m³
4. Find the coldest layer in each profile (hint: use `.min()` with `dim='layer'`)
5. Count how many layers have density > 300 kg/m³ in each profile


In [ ]:
# Target Practice Exercise 1: Temperature between -5°C and 0°C
# Your code here:
# temp_mask = (ds['temperature'] > ???) & (ds['temperature'] < ???)
# moderate_temp_layers = ds.where(temp_mask, drop=True)

# Target Practice Exercise 2: Density > 250 kg/m³
# Your code here:
# dense_mask = ds['???'] > ???
# dense_layers = ds.where(dense_mask, drop=True)

# Target Practice Exercise 3: Temperature < -10°C OR density < 100 kg/m³
# Your code here:
# extreme_mask = (ds['temperature'] < ???) | (ds['density'] < ???)
# extreme_layers = ds.where(extreme_mask, drop=True)

# Target Practice Exercise 4: Coldest layer in each profile
# Your code here:
# coldest_temp = ds['temperature'].???(dim='layer')

# Target Practice Exercise 5: Count layers with density > 300 kg/m³
# Your code here:
# high_density_count = (ds['density'] > ???).sum(dim='layer')


## 🎯 Target Practice: Aggregations

Practice these aggregation operations:

1. Compute mean temperature across all layers for each profile
2. Find the maximum density in each profile
3. Compute standard deviation of density across layers
4. Find the minimum temperature in each profile
5. Count the number of layers in each profile (hint: use `.count()` or `.size`)


In [ ]:
# Target Practice Exercise 1: Mean temperature across layers
# Your code here:
# mean_temp = ds['temperature'].???(dim='layer')

# Target Practice Exercise 2: Maximum density in each profile
# Your code here:
# max_density = ds['density'].???(dim='layer')

# Target Practice Exercise 3: Standard deviation of density
# Your code here:
# std_density = ds['density'].???(dim='layer')

# Target Practice Exercise 4: Minimum temperature
# Your code here:
# min_temp = ds['temperature'].???(dim='layer')

# Target Practice Exercise 5: Count layers in each profile
# Your code here:
# layer_count = ds['density'].???(dim='layer')


## 🐛 Bug Hunt: Find and Fix the Errors

**Bug Hunt Pattern**: The code below has intentional bugs! Find and fix them. Each buggy code block has 2-3 errors.

**Hint**: Common errors include:
- Wrong dimension names
- Missing parameters (like `drop=True`)
- Incorrect method calls
- Logic errors


In [ ]:
# Bug Hunt Exercise 1: This code is supposed to compute mean density, but has 2 bugs
# Find and fix them:
# mean_density = ds['density'].mean(dim='time')  # Bug 1: wrong dimension
# print(f"Mean density: {mean_density.values[0]:.1f}")

# Bug Hunt Exercise 2: This code should filter for dense layers, but has 2 bugs
# Find and fix them:
# dense_mask = ds['temperature'] > 300  # Bug 1: wrong variable
# dense_layers = ds.where(dense_mask)   # Bug 2: missing drop=True

# Bug Hunt Exercise 3: This code should select the first location, but has 1 bug
# Find and fix it:
# location_name = ds.coords['location'].values[0]
# ds_site = ds.isel(location=location_name)  # Bug: should use .sel() not .isel()

# Bug Hunt Exercise 4: This code should compute max temperature, but has 1 bug
# Find and fix it:
# max_temp = ds['temperature'].max()  # Bug: missing dim parameter

# Bug Hunt Exercise 5: This code should find cold layers, but has 2 bugs
# Find and fix them:
# cold_mask = ds['density'] < -10  # Bug 1: wrong variable and wrong comparison
# cold_layers = ds.where(cold_mask)  # Bug 2: missing drop=True


## Summary

✅ **What we learned:**

1. **Selecting data**: `.sel()` for labels, `.isel()` for positions
2. **Filtering**: `.where()` with conditions to find specific layers/profiles
3. **Aggregations**: `.mean()`, `.max()`, `.min()`, `.sum()` across dimensions
4. **SWE calculation**: Sum of density × thickness over layers
5. **Weak layer identification**: Using conditions to find problematic layers
6. **Time series**: Analyzing changes over time with `.diff()`, `.mean()`, etc.
7. **NumPy integration**: `.values` to get raw arrays when needed

## Key Operations Cheat Sheet

```python
# Selection
ds.sel(location="VIR1A", time="2024-02-01")  # By label
ds.isel(time=0, layer=0)                      # By position

# Filtering
ds.where(ds['density'] > 300, drop=True)      # Condition

# Aggregations
ds['density'].mean(dim='layer')               # Mean over layers
ds['HS'].max(dim='time')                      # Max over time

# Time operations
ds['HS'].diff(dim='time')                     # Change over time
ds.resample(time='1D').mean()                 # Resample

# NumPy
ds['density'].values                          # Get array
```

## Next Steps

Ready to visualize your data? Move on to:
- **03_visualization.ipynb**: Create plots and visualizations

## Exercises

1. Select data for a specific date and print the density profile
2. Find all layers with temperature below -5°C
3. Compute the mean density for the top 3 layers across all time steps
4. Calculate how much the snow height changed between the first and last time step
5. Identify profiles that have at least one layer with density < 100 kg/m³
